## Motion+Sound Confusion Matrix

In [6]:
# Motion+Sound Time-Voting & Confusion Matrix Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from pathlib import Path

def load_probabilities(pid: int, model_version: int, base_dir: Path) -> (np.ndarray, np.ndarray):
    """
    Load y_true and probability matrix for a participant.
    """
    file_path = base_dir / f"{pid}/{pid}_quan_ver{model_version}.csv"
    df = pd.read_csv(file_path)
    class_names = [col for col in df.columns if col not in ['y_true','y_pred']]
    y_true = df['y_true'].values
    probs = df[class_names].values
    return y_true, probs, class_names

def time_voting(probs: np.ndarray, window_size: int) -> list:
    """
    Apply moving-window averaging on probability vectors.
    """
    queue = []
    preds = []
    for p in probs:
        queue.append(p)
        if len(queue) > window_size:
            queue.pop(0)
        avg = np.mean(queue, axis=0)
        preds.append(int(np.argmax(avg)))
    return preds

def plot_confusion(y_true, y_pred, class_names, window_sec: float, out_dir: Path, pid: int):
    """
    Plot and save confusion matrix percentages.
    """
    cm = confusion_matrix(y_true, y_pred, labels=class_names)
    cm_pct = (cm.astype(float) / cm.sum(axis=1, keepdims=True)) * 100
    cm_pct = np.nan_to_num(cm_pct)
    fig, ax = plt.subplots(figsize=(10,8))
    im = ax.imshow(cm_pct, cmap='Greens', vmin=0, vmax=100)
    ticks = np.arange(len(class_names))
    ax.set_xticks(ticks); ax.set_yticks(ticks)
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.set_yticklabels(class_names)
    thresh = cm_pct.max() / 2
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            color = 'white' if cm_pct[i,j] > thresh else 'black'
            ax.text(j, i, f"{cm_pct[i,j]:.1f}", ha='center', va='center', color=color)
    ax.set_title(f"Confusion Matrix (w={window_sec:.1f}s)")
    fig.colorbar(im, ax=ax)
    out_dir.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_dir / f"{pid}_w{window_sec:.1f}s_cm.png")
    plt.close(fig)

if __name__ == '__main__':
    BASE_DIR = Path("../../Results/Experiment_Result/Model_Preds/Multimodal")
    CM_DIR   = Path("../../Results/Experiment_Result/Confusion_Matrix/Multimodal")
    model_version = 36
    pid = 201

    y_true, probs, class_names = load_probabilities(pid, model_version, BASE_DIR)

    best_bacc = -np.inf
    best_w = None

    for w in range(1, 20):
        window_sec = w * 0.2
        pred_idx = time_voting(probs, window_size=w)
        y_pred = np.array([class_names[i] for i in pred_idx])
        bacc = balanced_accuracy_score(y_true, y_pred)
        print(f"w={window_sec:.1f}s  BA={bacc:.4f}")
        if bacc > best_bacc:
            best_bacc, best_w = bacc, window_sec
        plot_confusion(y_true, y_pred, class_names, window_sec, CM_DIR / str(pid), pid)

    print(f"Best Balanced Accuracy: {best_bacc:.4f} at w={best_w:.1f}s")

w=0.2s  BA=0.5937
w=0.4s  BA=0.5952
w=0.6s  BA=0.5961
w=0.8s  BA=0.5971
w=1.0s  BA=0.5987
w=1.2s  BA=0.6001
w=1.4s  BA=0.6019
w=1.6s  BA=0.6032
w=1.8s  BA=0.6042
w=2.0s  BA=0.6041
w=2.2s  BA=0.6056
w=2.4s  BA=0.6058
w=2.6s  BA=0.6087
w=2.8s  BA=0.6085
w=3.0s  BA=0.6096
w=3.2s  BA=0.6102
w=3.4s  BA=0.6122
w=3.6s  BA=0.6137
w=3.8s  BA=0.6150
Best Balanced Accuracy: 0.6150 at w=3.8s
